In [ ]:
from glob import glob as glob
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import pandas as pd

def get_date(file, year=2018):
    parts = file[len("logs/results-"):-4]
    return datetime(year, int(parts[0:2]), int(parts[2:4]), int(parts[4:6]), int(parts[6:8]))

In [ ]:
prev = None
dfs_l = []
for file in glob("logs/*.txt"):
    with open(file, "r") as r:
        rd = csv.reader(r, delimiter="\t", quotechar='"')
        lines_to_df = []
        for l in rd:
            if len(l) <4 : continue
            if l[3] == '0':
                prev = l
            else:
                prev[3] = l[3]
                lines_to_df.append(prev)
                lines_to_df.append(l)
        frame = pd.DataFrame(lines_to_df, 
                             columns=[
                                 'run', 'data', 
                                 'instance', 'records', 
                                 'insertion', 'ins_retrieve','retrieve'])
        frame[['records','insertion','ins_retrieve','retrieve']] = frame[['records','insertion','ins_retrieve','retrieve']].apply(pd.to_numeric)
        frame['date'] = get_date(file)
        dfs_l.append(frame)
results = pd.concat(dfs_l)
results.sample(10)

In [ ]:
aggregate = results.groupby(['data','instance','records']).mean()
flights = aggregate.loc['flights'].reset_index()
other = aggregate.loc['other'].reset_index()

In [ ]:
marg = 0

instance_dict = {
    '3.4.0-apoc': ['As strings, using APOC (3.4.0)', 'rv--'],
    '3.4.0-plain': ['As strings (3.4.0)', 'g^--'],
    '3.5.0-maps': ['As maps (3.5.0)', 'bo--'],
}

measures_dict = {
    'insertion': 'Insertion', 
    'ins_retrieve': 'Retrieval'
}

def plot(data, measures):
    for measure in measures:
        plt.figure(figsize=(10,6), frameon=False)
        instances = data.instance.unique()
        sns.barplot(x='records', y=measure, data=data,hue='instance')
        plt.legend(loc='upper left', frameon=True)
        plt.margins(0.01,0.02)
#        plt.xlim(-marg, 1_000_000 + marg)
        plt.xlabel('Records')
        plt.ylabel('Seconds elapsed')
        plt.yscale('log')
        plt.title(measures_dict[measure])
        plt.grid(True, alpha=0.5)
        plt.tight_layout()
        plt.savefig('images/'+measure)
plot(flights, ['insertion', 'ins_retrieve'])